## Desafio DWE 2025 - Transformação Camada Silver
**Aqui serão feitas as transformações de modelagem dimensional e de dados, como:**
- Transformar tabelas e transferir dados para que as relações fiquem de acordo com o esquema conceitual
- Transformações de colunas com dados que atendam a estrutura de negócio e do modelo
- Salvamento de arquivos atendendo o tipo de carga (completa ou incremental)

In [0]:
import gc
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder \
    .appName("Carga e Transformação Silver") \
    .config("spark.sql.shuffle.partitions", "200")  \
    .config("spark.sql.files.maxPartitionBytes", "128MB") \
    .config("spark.sql.parquet.compression.codec", "snappy") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()


# Diretórios de trabalho 
br_dim_path = '/mnt/desafio-dwe-25/lhdw/bronze/dim'
br_fato_path = '/mnt/desafio-dwe-25/lhdw/bronze/fato'

sv_dim_path = '/mnt/desafio-dwe-25/lhdw/silver/dim'
sv_fato_path = '/mnt/desafio-dwe-25/lhdw/silver/fato'

### Dimensão - Categoria

In [0]:
br_file = f'{br_dim_path}/CATEGORIA'

# Carregamento de Categoria na Camada Bronze
df_categoria = spark.read.format("delta").load(br_file)

# Checagem de dados e tipagem das colunas
df_categoria.show(5)
df_categoria.printSchema()

+-----------+-------------+
|CategoriaID|NomeCategoria|
+-----------+-------------+
|          1|  Confections|
|          2|   Shell fish|
|          3|      Cereals|
|          4|        Dairy|
|          5|    Beverages|
+-----------+-------------+
only showing top 5 rows

root
 |-- CategoriaID: integer (nullable = true)
 |-- NomeCategoria: string (nullable = true)



In [0]:
silver_path = f'{sv_dim_path}/CATEGORIA'

# Inserção da coluna de atualização do dado
df_categoria_silver = df_categoria \
    .withColumn("DATA_ATUALIZACAO", current_timestamp())

df_categoria_silver.show(5)
df_categoria_silver.printSchema()

# Salvando a tabela na camada Silver
df_categoria_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(silver_path)

# Conferindo o diretório Silver da tabela
print(f"\n Tabela Salva em {silver_path}:")
spark.read.format("delta").option("header", "true").load(silver_path).show(5)


+-----------+-------------+--------------------+
|CategoriaID|NomeCategoria|    DATA_ATUALIZACAO|
+-----------+-------------+--------------------+
|          1|  Confections|2025-02-21 18:30:...|
|          2|   Shell fish|2025-02-21 18:30:...|
|          3|      Cereals|2025-02-21 18:30:...|
|          4|        Dairy|2025-02-21 18:30:...|
|          5|    Beverages|2025-02-21 18:30:...|
+-----------+-------------+--------------------+
only showing top 5 rows

root
 |-- CategoriaID: integer (nullable = true)
 |-- NomeCategoria: string (nullable = true)
 |-- DATA_ATUALIZACAO: timestamp (nullable = false)


 Tabela Salva em /mnt/desafio-dwe-25/lhdw/silver/dim/CATEGORIA:
+-----------+-------------+--------------------+
|CategoriaID|NomeCategoria|    DATA_ATUALIZACAO|
+-----------+-------------+--------------------+
|          1|  Confections|2025-02-21 18:31:...|
|          2|   Shell fish|2025-02-21 18:31:...|
|          3|      Cereals|2025-02-21 18:31:...|
|          4|        Dairy|2

In [0]:
# Limpeza de cache
df_categoria.unpersist()
df_categoria_silver.unpersist()

Out[5]: DataFrame[CategoriaID: int, NomeCategoria: string, DATA_ATUALIZACAO: timestamp]

### Dimensão - Produto

In [0]:
br_file = f'{br_dim_path}/PRODUTO'

# Carregamento de Produto da Camada Bronze
df_produto = spark.read.format("delta").load(br_file)

# Checagem de dados e tipagem das colunas
df_produto.show(5)
df_produto.printSchema()

+---------+--------------------+-------+-----------+------+--------------------+-----------+---------+------------+
|ProdutoID|         ProdutoNome|  Preco|CategoriaID|Classe|        DataCadastro|Resistencia|EAlergico|ValidadeDias|
+---------+--------------------+-------+-----------+------+--------------------+-----------+---------+------------+
|        1| Flour - Whole Wheat|74.2988|          3|Medium|2018-02-16 08:21:...|    Durable|  Unknown|         0.0|
|        2|Cookie Chocolate ...|91.2329|          3|Medium|2017-02-12 11:39:...|    Unknown|  Unknown|         0.0|
|        3|  Onions - Cippolini| 9.1379|          9|Medium|2018-03-15 08:11:...|       Weak|    False|       111.0|
|        4|Sauce - Gravy, Au...|54.3055|          9|Medium|2017-07-16 00:46:...|    Durable|  Unknown|         0.0|
|        5|Artichokes - Jeru...|65.4771|          2|   Low|2017-08-16 14:13:...|    Durable|     True|        27.0|
+---------+--------------------+-------+-----------+------+-------------

In [0]:
silver_path = f'{sv_dim_path}/PRODUTO'

# Inserção da coluna de atualização do dado
df_produto_silver = df_produto \
    .withColumn("DATA_ATUALIZACAO", current_timestamp())

df_produto_silver.show(5)
df_produto_silver.printSchema()

# Salvando a tabela na camada Silver
df_produto_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(silver_path)

# Conferindo o diretório Silver da tabela
print(f"\n Tabela Salva em {silver_path}:")
spark.read.format("delta").option("header", "true").load(silver_path).show(5)

+---------+--------------------+-------+-----------+------+--------------------+-----------+---------+------------+--------------------+
|ProdutoID|         ProdutoNome|  Preco|CategoriaID|Classe|        DataCadastro|Resistencia|EAlergico|ValidadeDias|    DATA_ATUALIZACAO|
+---------+--------------------+-------+-----------+------+--------------------+-----------+---------+------------+--------------------+
|        1| Flour - Whole Wheat|74.2988|          3|Medium|2018-02-16 08:21:...|    Durable|  Unknown|         0.0|2025-02-21 18:31:...|
|        2|Cookie Chocolate ...|91.2329|          3|Medium|2017-02-12 11:39:...|    Unknown|  Unknown|         0.0|2025-02-21 18:31:...|
|        3|  Onions - Cippolini| 9.1379|          9|Medium|2018-03-15 08:11:...|       Weak|    False|       111.0|2025-02-21 18:31:...|
|        4|Sauce - Gravy, Au...|54.3055|          9|Medium|2017-07-16 00:46:...|    Durable|  Unknown|         0.0|2025-02-21 18:31:...|
|        5|Artichokes - Jeru...|65.4771| 

In [0]:
# Liberação de cache
df_produto.unpersist()
df_produto_silver.unpersist()

Out[8]: DataFrame[ProdutoID: int, ProdutoNome: string, Preco: double, CategoriaID: int, Classe: string, DataCadastro: string, Resistencia: string, EAlergico: string, ValidadeDias: double, DATA_ATUALIZACAO: timestamp]

### Dimensão - Vendedor

In [0]:
br_file = f'{br_dim_path}/VENDEDOR'

# Carregamento de Vendedor da Camada Bronze
df_vendedor = spark.read.format("delta").load(br_file)

# Checagem de dados e tipagem das colunas
df_vendedor.show(5)
df_vendedor.printSchema()

+----------+------------+----------+----------+--------------+------+--------+------------+
|VendedorID|PrimeiroNome|NomeDoMeio|UltimoNome|DataNascimento|Genero|CidadeID|DataAdmissao|
+----------+------------+----------+----------+--------------+------+--------+------------+
|         1|      Nicole|         T|    Fuller|    1981-03-07|     F|      80|  2011-06-20|
|         2|   Christine|         W|    Palmer|    1968-01-25|     F|       4|  2011-04-27|
|         3|       Pablo|         Y|     Cline|    1963-02-09|     M|      70|  2012-03-30|
|         4|     Darnell|         O|   Nielsen|    1989-02-06|     M|      39|  2014-03-06|
|         5|     Desiree|         L|    Stuart|    1963-05-03|     F|      23|  2014-11-16|
+----------+------------+----------+----------+--------------+------+--------+------------+
only showing top 5 rows

root
 |-- VendedorID: integer (nullable = true)
 |-- PrimeiroNome: string (nullable = true)
 |-- NomeDoMeio: string (nullable = true)
 |-- UltimoNo

In [0]:
silver_path = f'{sv_dim_path}/VENDEDOR'

# Inserção da coluna de atualização do dado e concatenação do nome do Vendedor
df_vendedor_silver = df_vendedor \
    .withColumn("DATA_ATUALIZACAO", current_timestamp()) \
    .withColumn("PrimeiroNome", concat_ws(" ", col("PrimeiroNome"), col("NomeDoMeio"), col("UltimoNome"))) \
    .withColumnRenamed("PrimeiroNome", "NomeVendedor") \
    .drop("NomeDoMeio", "UltimoNome")

df_vendedor_silver.show(5)
df_vendedor_silver.printSchema()

# Salvando a tabela na camada Silver
df_vendedor_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(silver_path)

# Conferindo o diretório Silver da tabela
print(f"\n Tabela Salva em {silver_path}:")
spark.read.format("delta").option("header", "true").load(silver_path).show(5)

+----------+------------------+--------------+------+--------+------------+--------------------+
|VendedorID|      NomeVendedor|DataNascimento|Genero|CidadeID|DataAdmissao|    DATA_ATUALIZACAO|
+----------+------------------+--------------+------+--------+------------+--------------------+
|         1|   Nicole T Fuller|    1981-03-07|     F|      80|  2011-06-20|2025-02-21 18:43:...|
|         2|Christine W Palmer|    1968-01-25|     F|       4|  2011-04-27|2025-02-21 18:43:...|
|         3|     Pablo Y Cline|    1963-02-09|     M|      70|  2012-03-30|2025-02-21 18:43:...|
|         4| Darnell O Nielsen|    1989-02-06|     M|      39|  2014-03-06|2025-02-21 18:43:...|
|         5|  Desiree L Stuart|    1963-05-03|     F|      23|  2014-11-16|2025-02-21 18:43:...|
+----------+------------------+--------------+------+--------+------------+--------------------+
only showing top 5 rows

root
 |-- VendedorID: integer (nullable = true)
 |-- NomeVendedor: string (nullable = false)
 |-- Data

In [0]:
# Liberação de cache
df_vendedor.unpersist()
df_vendedor_silver.unpersist()

Out[18]: DataFrame[VendedorID: int, NomeVendedor: string, DataNascimento: date, Genero: string, CidadeID: int, DataAdmissao: date, DATA_ATUALIZACAO: timestamp]

### Dimensão - Pais

In [0]:
br_file = f'{br_dim_path}/PAIS'

# Carregamento de País da Camada Bronze
df_pais = spark.read.format("delta").load(br_file)

# Checagem de dados e tipagem das colunas
df_pais.show(5)
df_pais.printSchema()

+------+--------+---------+
|PaisID|PaisNome|SiglaPais|
+------+--------+---------+
|     1| Armenia|       AN|
|     2|  Canada|       FO|
|     3|  Belize|       MK|
|     4|  Uganda|       LV|
|     5|Thailand|       VI|
+------+--------+---------+
only showing top 5 rows

root
 |-- PaisID: integer (nullable = true)
 |-- PaisNome: string (nullable = true)
 |-- SiglaPais: string (nullable = true)



In [0]:
silver_path = f'{sv_dim_path}/PAIS'

# Inserção da coluna de atualização do dado
df_pais_silver = df_pais \
    .withColumn("DATA_ATUALIZACAO", current_timestamp())

df_pais_silver.show(5)
df_pais_silver.printSchema()

# Salvando a tabela na camada Silver
df_pais_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(silver_path)

# Conferindo o diretório Silver da tabela
print(f"\n Tabela Salva em {silver_path}:")
spark.read.format("delta").option("header", "true").load(silver_path).show(5)

+------+--------+---------+--------------------+
|PaisID|PaisNome|SiglaPais|    DATA_ATUALIZACAO|
+------+--------+---------+--------------------+
|     1| Armenia|       AN|2025-02-21 18:43:...|
|     2|  Canada|       FO|2025-02-21 18:43:...|
|     3|  Belize|       MK|2025-02-21 18:43:...|
|     4|  Uganda|       LV|2025-02-21 18:43:...|
|     5|Thailand|       VI|2025-02-21 18:43:...|
+------+--------+---------+--------------------+
only showing top 5 rows

root
 |-- PaisID: integer (nullable = true)
 |-- PaisNome: string (nullable = true)
 |-- SiglaPais: string (nullable = true)
 |-- DATA_ATUALIZACAO: timestamp (nullable = false)


 Tabela Salva em /mnt/desafio-dwe-25/lhdw/silver/dim/PAIS:
+------+--------+---------+--------------------+
|PaisID|PaisNome|SiglaPais|    DATA_ATUALIZACAO|
+------+--------+---------+--------------------+
|     1| Armenia|       AN|2025-02-21 18:43:...|
|     2|  Canada|       FO|2025-02-21 18:43:...|
|     3|  Belize|       MK|2025-02-21 18:43:...|
| 

In [0]:
# Liberação de cache
df_pais.unpersist()
df_pais_silver.unpersist()

Out[26]: DataFrame[PaisID: int, PaisNome: string, SiglaPais: string, DATA_ATUALIZACAO: timestamp]

### Dimensão - Cidade

In [0]:
br_file = f'{br_dim_path}/CIDADE'

# Carregamento de Cidade na Camada Bronze
df_cidade = spark.read.format("delta").load(br_file)

# Checagem de dados e tipagem das colunas
df_cidade.show(5)
df_cidade.printSchema()

+--------+--------------+-----+------+
|CidadeID|    NomeCidade|  Cep|PaisID|
+--------+--------------+-----+------+
|       1|        Dayton|80563|    32|
|       2|       Buffalo|17420|    32|
|       3|       Chicago|44751|    32|
|       4|       Fremont|20641|    32|
|       5|Virginia Beach|62389|    32|
+--------+--------------+-----+------+
only showing top 5 rows

root
 |-- CidadeID: integer (nullable = true)
 |-- NomeCidade: string (nullable = true)
 |-- Cep: string (nullable = true)
 |-- PaisID: string (nullable = true)



In [0]:
silver_path = f'{sv_dim_path}/CIDADE'

# Inserção da coluna de atualização do dado
df_cidade_silver = df_cidade \
    .withColumn("DATA_ATUALIZACAO", current_timestamp())

df_cidade_silver.show(5)
df_cidade_silver.printSchema()

# Salvando a tabela na camada Silver
df_cidade_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(silver_path)

# Conferindo o diretório Silver da tabela
print(f"\n Tabela Salva em {silver_path}:")
spark.read.format("delta").option("header", "true").load(silver_path).show(5)

+--------+--------------+-----+------+--------------------+
|CidadeID|    NomeCidade|  Cep|PaisID|    DATA_ATUALIZACAO|
+--------+--------------+-----+------+--------------------+
|       1|        Dayton|80563|    32|2025-02-21 18:45:...|
|       2|       Buffalo|17420|    32|2025-02-21 18:45:...|
|       3|       Chicago|44751|    32|2025-02-21 18:45:...|
|       4|       Fremont|20641|    32|2025-02-21 18:45:...|
|       5|Virginia Beach|62389|    32|2025-02-21 18:45:...|
+--------+--------------+-----+------+--------------------+
only showing top 5 rows

root
 |-- CidadeID: integer (nullable = true)
 |-- NomeCidade: string (nullable = true)
 |-- Cep: string (nullable = true)
 |-- PaisID: string (nullable = true)
 |-- DATA_ATUALIZACAO: timestamp (nullable = false)


 Tabela Salva em /mnt/desafio-dwe-25/lhdw/silver/dim/CIDADE:
+--------+--------------+-----+------+--------------------+
|CidadeID|    NomeCidade|  Cep|PaisID|    DATA_ATUALIZACAO|
+--------+--------------+-----+------+-

In [0]:
# Liberação de cache
df_cidade.unpersist()
df_cidade_silver.unpersist()

Out[29]: DataFrame[CidadeID: int, NomeCidade: string, Cep: string, PaisID: string, DATA_ATUALIZACAO: timestamp]

### Dimensão - Cliente

In [0]:
br_file = f'{br_dim_path}/CLIENTE'

# Carregamento de Cliente na Camada Bronze
df_cliente = spark.read.format("delta").load(br_file)

# Checagem de dados e tipagem das colunas
df_cliente.show(5)
df_cliente.printSchema()

+---------+------------+----------+----------+--------+--------------------+
|ClienteID|PrimeiroNome|NomeDoMeio|UltimoNome|CidadeID|            Endereco|
+---------+------------+----------+----------+--------+--------------------+
|        1|    Stefanie|         Y|      Frye|      79|       97 Oak Avenue|
|        2|       Sandy|         T|     Kirby|      96|52 White First Fr...|
|        3|         Lee|         T|     Zhang|      55|921 White Fabien ...|
|        4|      Regina|         S|     Avery|      40|       75 Old Avenue|
|        5|      Daniel|         S|    Mccann|       2|283 South Green H...|
+---------+------------+----------+----------+--------+--------------------+
only showing top 5 rows

root
 |-- ClienteID: integer (nullable = true)
 |-- PrimeiroNome: string (nullable = true)
 |-- NomeDoMeio: string (nullable = true)
 |-- UltimoNome: string (nullable = true)
 |-- CidadeID: integer (nullable = true)
 |-- Endereco: string (nullable = true)



In [0]:
silver_path = f'{sv_dim_path}/CLIENTE'

# Inserção da coluna de atualização do dado e concatenação do nome do cliente 
df_cliente_silver = df_cliente \
    .withColumn("DATA_ATUALIZACAO", current_timestamp()) \
    .withColumn("PrimeiroNome", concat_ws(" ", col("PrimeiroNome"), col("NomeDoMeio"), col("UltimoNome"))) \
    .withColumnRenamed("PrimeiroNome", "NomeCliente") \
    .drop("NomeDoMeio", "UltimoNome")

df_cliente_silver.show(5)
df_cliente_silver.printSchema()

# Salvando a tabela na camada Silver
df_cliente_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(silver_path)

# Conferindo o diretório Silver da tabela
print(f"\n Tabela Salva em {silver_path}:")
spark.read.format("delta").option("header", "true").load(silver_path).show(5)

+---------+---------------+--------+--------------------+--------------------+
|ClienteID|    NomeCliente|CidadeID|            Endereco|    DATA_ATUALIZACAO|
+---------+---------------+--------+--------------------+--------------------+
|        1|Stefanie Y Frye|      79|       97 Oak Avenue|2025-02-21 18:46:...|
|        2|  Sandy T Kirby|      96|52 White First Fr...|2025-02-21 18:46:...|
|        3|    Lee T Zhang|      55|921 White Fabien ...|2025-02-21 18:46:...|
|        4| Regina S Avery|      40|       75 Old Avenue|2025-02-21 18:46:...|
|        5|Daniel S Mccann|       2|283 South Green H...|2025-02-21 18:46:...|
+---------+---------------+--------+--------------------+--------------------+
only showing top 5 rows

root
 |-- ClienteID: integer (nullable = true)
 |-- NomeCliente: string (nullable = false)
 |-- CidadeID: integer (nullable = true)
 |-- Endereco: string (nullable = true)
 |-- DATA_ATUALIZACAO: timestamp (nullable = false)


 Tabela Salva em /mnt/desafio-dwe-25/l

In [0]:
# Liberação de cache
df_cliente.unpersist()
df_cliente_silver.unpersist()

Out[33]: DataFrame[ClienteID: int, NomeCliente: string, CidadeID: int, Endereco: string, DATA_ATUALIZACAO: timestamp]

### Fato - Vendas

In [0]:
br_file = f'{br_fato_path}/VENDAS'
silver_path = f'{sv_fato_path}/VENDAS'

# Caso já existem registros de Vendas na Silver, selecionamos os registros novos da Bronze
# Caso contrário, usamos todos os registros da Bronze
try: 
    df_vendas_silver = spark.read.format("delta").load(silver_path) 
    ultimo_id_venda = df_vendas_silver.agg(max(col("VendasID"))).collect()[0][0]

    df_vendas = spark.read.format("delta").load(br_file) \
        .filter(col("VendasID") > ultimo_num_venda) 
except:
    df_vendas = spark.read.format("delta").load(br_file)

# Checagem de dados e tipagem das colunas
df_vendas.show(5)
df_vendas.printSchema()

+--------+----------+---------+---------+----------+--------+----------+----------+--------------------+
|VendasID|VendedorID|ClienteID|ProdutoID|Quantidade|Desconto|PrecoTotal| DataVenda|     NumeroTransacao|
+--------+----------+---------+---------+----------+--------+----------+----------+--------------------+
| 6207624|        16|     1401|        7|         1|     0.1|       0.0|2018-01-01|JMAICVALK5QLQPFJZMPQ|
| 2266176|         1|    28628|       79|         8|     0.1|       0.0|2018-01-01|XJ2AASIGXZFUOKQ1HMLJ|
| 6510351|        12|    64721|      411|        17|     0.0|       0.0|2018-01-01|AISH11C7JF8C4YRZ7PLQ|
| 1972817|         3|    84022|      425|        22|     0.0|       0.0|2018-01-01|SQQ4UEB0ML0QPNLTNLG9|
| 5564470|        17|    10737|      101|         3|     0.0|       0.0|2018-01-01|1AVUBWY1L2JL8ZXG5KE2|
+--------+----------+---------+---------+----------+--------+----------+----------+--------------------+
only showing top 5 rows

root
 |-- VendasID: integer (n

In [0]:
produto_path = f'{sv_dim_path}/PRODUTO'

# Resgatando a coluna Preco da tabela Produto em Silver
df_produto_silver = spark.read.format("delta") \
    .load(produto_path) \
    .select("ProdutoID", "Preco")

# Fazendo inner join entre Vendas e Produto para ter acesso ao Preco
df_vendas_preco = df_vendas.join(df_produto_silver, on="ProdutoID", how="inner")

# Calculando o PrecoTotal para cada registro de venda considerando o Preco unitário,
# a Quantidade e o desconto aplicado de acordo com a tabela Vendas
df_vendas_preco = df_vendas_preco \
    .withColumn("PrecoTotal", col("Preco") * col("Quantidade") * (1 - col("Desconto")))

# Limpeza das colunas de join, inserção de Data de Atualização, colunas de partição
# e exclusão da tabela NumeroTransacao
df_vendas_silver = df_vendas_preco.drop("Preco") \
    .withColumn("DATA_ATUALIZACAO", current_timestamp()) \
    .withColumn("ANO", year(col("DataVenda"))) \
    .withColumn("MES", month(col("DataVenda"))) \
    .drop("NumeroTransacao")

# Conferindo dados e schema
df_vendas_silver.show(5)
df_vendas_silver.printSchema()

# Salvando a tabela na camada Silver
df_vendas_silver.write \
    .format("delta") \
    .mode("append") \
    .partitionBy("ANO", "MES") \
    .option("mergeSchema", "true") \
    .save(silver_path)

# Conferindo o diretório Silver da tabela
print(f"\n Tabela Salva em {silver_path}:")
spark.read.format("delta").option("header", "true").load(silver_path).show(5)

+---------+--------+----------+---------+----------+--------+----------+----------+--------------------+----+---+
|ProdutoID|VendasID|VendedorID|ClienteID|Quantidade|Desconto|PrecoTotal| DataVenda|    DATA_ATUALIZACAO| ANO|MES|
+---------+--------+----------+---------+----------+--------+----------+----------+--------------------+----+---+
|      200| 4387325|         2|    41633|        11|     0.2|   97.9396|2018-03-16|2025-02-21 18:48:...|2018|  3|
|      384| 3158417|         5|     6322|         2|     0.2|  90.43152|2018-03-16|2025-02-21 18:48:...|2018|  3|
|      371| 3452246|        22|    48982|        13|     0.0| 1071.2286|2018-03-16|2025-02-21 18:48:...|2018|  3|
|      344| 1141829|         1|    32265|         9|     0.0|  571.8501|2018-03-16|2025-02-21 18:48:...|2018|  3|
|      399|  617495|        23|    53593|        14|     0.2| 570.20768|2018-03-16|2025-02-21 18:48:...|2018|  3|
+---------+--------+----------+---------+----------+--------+----------+----------+-----

In [0]:
# Liberação do cache
df_vendas.unpersist()
df_vendas_silver.unpersist()
df_vendas_preco.unpersist()
df_produto_silver.unpersist()
gc.collect()

Out[37]: DataFrame[ProdutoID: int, Preco: double]